In [7]:
import warnings
warnings.filterwarnings("ignore")

import os
import cv2

import numpy as np
import pandas as pd
import itertools
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Input, UpSampling2D, BatchNormalization, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'cv2'

In [8]:
# Set the tumor categories
tumors = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# Function to display 5 images from each folder side by side
def display_images_from_folders(folder_paths, n=5):
    fig, axs = plt.subplots(len(folder_paths), n, figsize=(8, 3))
    for i, folder_path in enumerate(folder_paths):
        image_files = os.listdir(folder_path)
        for j in range(n):
            image_path = os.path.join(folder_path, image_files[j])
            image = cv2.imread(image_path)
            axs[j].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axs[j].axis('off')
            axs[j].set_title(f'{os.path.basename(folder_path)}')
    plt.tight_layout()
    plt.show()

def load_images(dir, title):
  folder = dir
  data = []
  labels = []

  # Loop through each tumor category
  for tumor in tumors:
    # Set the path to the tumor category folder
    tumor_folder = os.path.join(folder, tumor)

    # Display 5 images from the folder
    display_images_from_folders([tumor_folder])

    # Get the list of image filenames in the folder
    image_files = os.listdir(tumor_folder)

    # Loop through each image file
    for image_file in image_files:
        # Read the image
        image_path = os.path.join(tumor_folder, image_file)
        image = cv2.imread(image_path)

        # Resize the image to a fixed size (e.g., 48x48 pixels)
        resized_image = cv2.resize(image, (48, 48))

        # Normalize the image data
        normalized_image = resized_image / 255.0

        # Append the image data and label to the lists
        data.append(normalized_image)
        labels.append(tumor)

  # Convert the data and labels lists to NumPy arrays
  data = np.array(data)
  labels = np.array(labels)

  # Display the bar chart showing the number of images in each class
  num_images_per_class = [len(data[labels == tumor]) for tumor in tumors]
  plt.bar(tumors, num_images_per_class)
  plt.xlabel('Tumor')
  plt.ylabel('Number of Images')
  plt.title(f'Images in {title} set')
  plt.show()

  return data, labels

In [4]:
X_train, y_train = load_images('/content/drive/MyDrive/June 2023/Brain Tumor Classification/Training', 'Training')

NameError: name 'plt' is not defined

In [ ]:
X_test, y_test = load_images('/content/drive/MyDrive/June 2023/Brain Tumor Classification/Testing', 'Testing')

In [ ]:
# Getting the percentage of frequency of classes in training and testing sets
f, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
pd.DataFrame(y_train).value_counts().plot.pie(autopct='%1.1f%%',ax=ax[0])
ax[0].set_title('Training set: Tumor distribution')
pd.DataFrame(y_test).value_counts().plot.pie(autopct='%1.1f%%',ax=ax[1])
ax[1].set_title('Testing set: Tumor distribution')
plt.show()

### CNN with data augmentation

In [ ]:
y_train_new = []
for i in y_train:
    y_train_new.append(tumors.index(i))
y_train_1 = y_train_new
y_train_1 = tf.keras.utils.to_categorical(y_train_1)


y_test_new = []
for i in y_test:
    y_test_new.append(tumors.index(i))
y_test_1 = y_test_new
y_test_1 = tf.keras.utils.to_categorical(y_test_1)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    cm = np.round(cm,2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
# Create the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # 4 types of brain tumors

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation to improve generalization
datagen = ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

# Train the model
batch_size = 32
epochs = 25
history_1 = model.fit(datagen.flow(X_train, y_train_1, batch_size=batch_size),
                      steps_per_epoch=len(X_train) / batch_size,
                      epochs=epochs,
                      validation_data=(X_test, y_test_1))

# Model summary
print(model.summary())

In [ ]:
y_pred_encoded = model.predict(X_test)
y_pred = np.argmax(y_pred_encoded,axis=1)
print('Testing accuracy: ', accuracy_score(y_pred, np.argmax(y_test_1, axis=1)))

cnn_confusion_mtx = confusion_matrix(np.argmax(y_test_1, axis=1), y_pred)
cm = plot_confusion_matrix(cnn_confusion_mtx, classes = tumors, normalize=False)

### GAN

In [ ]:
width, height, channel = 48, 48, 3

def sample_images(generator, noise, subplots, figsize=(22,8), save=False):
    generated_images = generator.predict(noise)
    plt.figure(figsize=figsize)

    for i, image in enumerate(generated_images):
        plt.subplot(subplots[0], subplots[1], i+1)
        if channel == 1:
            plt.imshow(image.reshape((width, height)), cmap='gray')
        else:
            plt.imshow(image.reshape((width, height, channel)))
        if save == True:
            img_name = "gen" + str(i)
            plt.savefig(img_name)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Load and preprocess your data (assuming X contains the images)
# Replace this with your data loading and preprocessing code
X = X_train # Shape: (number of records, 48, 48, 3)

# Normalize images to the range [-1, 1]
X = (X.astype(np.float32) - 127.5) / 127.5

# Define the Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128 * 12 * 12, input_dim=latent_dim))
    model.add(Reshape((12, 12, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(3, kernel_size=3, padding="same", activation="tanh"))
    return model

# Define the Discriminator model
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(4, activation="softmax"))
    return model

# Define hyperparameters
latent_dim = 100
img_shape = (48, 48, 3)

In [ ]:
# Build and compile the Discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build the Generator
generator = build_generator(latent_dim)

# The combined GAN model: Generator -> Discriminator
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)

combined = Model(z, validity)
combined.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5))

# Model summary
print(generator.summary())
print(discriminator.summary())
print(combined.summary())

# Training loop
epochs = 25
batch_size = 32
best_acc = 0.8

for epoch in range(epochs):
    # Select a random batch of real images
    idx = np.random.randint(0, X.shape[0], batch_size)
    real_imgs = X[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)

    # Train the Discriminator
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 4)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 4)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 4)))

    # Print progress
    print(f"Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Acc.: {100 * d_loss[1]}, Generator Loss: {g_loss}")

    if d_loss[1] >= best_acc:
      print(f"Reached desired accuracy ({best_acc}), stopping training.")
      break

In [ ]:
#Generation
noise = np.random.normal(0, 1, size=(100, latent_dim))
sample_images(generator, noise, (10,10), (24,20))

In [ ]:
fig, axs = plt.subplots(figsize=(10,5))

sns.distplot(X_train, label='Real Images', hist=True, color='#fc0328', ax=axs)
sns.distplot(fake_imgs, label='Generated Images', hist=True, color='#0c06c7', ax=axs)
axs.legend(loc='upper right')

plt.show()

### GAN with data augmentation

In [6]:
# Build and compile the Generator
generator_augment = build_generator(latent_dim)
generator_augment.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5))

# Build and compile the Discriminator
discriminator_augment = build_discriminator(img_shape)
discriminator_augment.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# The combined GAN model: Generator -> Discriminator
z = Input(shape=(latent_dim,))
img = generator_augment(z)
discriminator_augment.trainable = False
validity = discriminator_augment(img)

combined_augment = Model(z, validity)
combined_augment.compile(loss='categorical_crossentropy', optimizer=Adam(0.0002, 0.5))

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Model summary
print(generator_augment.summary())
print(discriminator_augment.summary())
print(combined_augment.summary())

epochs = 25
batch_size = 32

for epoch in range(epochs):
    # Select a batch of real images
    idx = np.random.randint(0, X.shape[0], batch_size)
    real_imgs = X[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator_augment.predict(noise)

    # Augment the batch of real images
    real_imgs_augmented = datagen.flow(real_imgs, batch_size=batch_size, shuffle=False).next()

    # Train the Discriminator on real and fake images
    real_labels = np.ones((batch_size, 4))
    fake_labels = np.zeros((batch_size, 4))
    d_loss_real = discriminator_augment.train_on_batch(real_imgs_augmented, real_labels)
    d_loss_fake = discriminator_augment.train_on_batch(fake_imgs, fake_labels)

    # Train the Generator to fool the Discriminator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 4))
    g_loss = combined_augment.train_on_batch(noise, valid_labels)

    # Print progress
    print(f"Epoch {epoch}, D Loss Real: {d_loss_real[0]}, Training Acc.: {100 * d_loss_real[1]}, D Loss Fake: {d_loss_fake[0]}, Testing Acc.: {100 * d_loss_fake[1]}, G Loss: {g_loss}")

NameError: name 'build_generator' is not defined

In [ ]:
#Generation
noise = np.random.normal(0, 1, size=(100, latent_dim))
sample_images(generator_augment, noise, (10,10), (24,20))

In [5]:
fig, axs = plt.subplots(figsize=(10,5))

sns.distplot(X, label='Real Images', hist=True, color='#fc0328', ax=axs)
sns.distplot(fake_imgs, label='Generated Images', hist=True, color='#0c06c7', ax=axs)
axs.legend(loc='upper right')

plt.show()

NameError: name 'plt' is not defined